## Electronic markets project - exo2_2 and 2_3

### Loading libraries and initializing parameters

In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [120]:
x0 = 10e4
sigma = 0.20
T = 1
eta = 1.1 * 1e-4
lambdas = [85e-4, 1e-1 , 0.5, 1, 3, 8]

### Creating optimal strategy functions

In [88]:
def percentage_of_volume_opti_strat(t, x0, sigma, lamb, eta):
    v = np.sqrt((lamb * (x0 ** 2) * (sigma ** 2)) / (3 * eta))
    return x0 - v * t 

### Percentage of volume - optimal strategy

#### Varying risk aversion

In [89]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)

for lamb in lambdas:
    xt = [max(percentage_of_volume_opti_strat(t, x0, sigma, lamb, eta), 0) for t in t_values]

    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'lambda': str(lamb)})
    df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='lambda', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "lambda": "Risk Aversion"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

#### Varying temporary market impact

In [90]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)
etas = [1e-6, 1e-5, 1e-4, 5e-4, 1e-3]
lamb = 0.1

for eta in etas:
    xt = [max(percentage_of_volume_opti_strat(t, x0, sigma, lamb, eta), 0) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'eta': str(eta)})
    df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='eta', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "eta": "Market Impact"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

### Percentage of volume - efficient frontier

In [121]:
def calculate_POV_expectation(lbd_values, eta, sigma, x0, T):
    v = np.sqrt((lbd_values * (x0 ** 2) * (sigma ** 2)) / (3 * eta))
    return -1 * eta * x0 * v 

def calculate_POV_variance(lbd_values, eta, sigma, x0, T):
    v = np.sqrt((lbd_values * (x0 ** 2) * (sigma ** 2)) / (3 * eta))
    return (x0 ** 3) * (sigma ** 2) / (3 * v)

lbd_values = np.linspace(1e-4, 1, num=1000)

expectation = calculate_POV_expectation(lbd_values, eta, sigma, x0, T)
variance = calculate_POV_variance(lbd_values, eta, sigma, x0, T)

df = pd.DataFrame({'E': np.abs(expectation) / 100, 'V': np.abs(variance) / 10000, 'lambda': lbd_values})

fig = px.scatter(df, x='V', y='E', color='lambda', template='plotly_white', 
                 labels={"V": "Variance", "E": "Abs(Expectation)", "lambda": "𝛌"}, color_continuous_scale='Purp')
fig.update_layout(width=900, height=300)
fig.update_layout(xaxis=dict(range=[800, 14000]))
fig.show()

In [122]:
def calculate_IS_expectation(k, eta, sigma, x0, T):
    return -1 * ((((k * x0) / (np.sinh(k * T)))**2) * eta) * ((2 * k * T + np.sinh(2 * k * T)) / (4 * k))

def calculate_IS_variance(k, sigma, x0, T):
    return (((sigma * x0) / (np.sinh(k * T)))**2) * ((np.sinh(2 * k * T) - (2 * k * T)) / (4 * k))

lbd_values = np.linspace(1e-4, 1, num=1000)

expectation_POV = calculate_POV_expectation(lbd_values, eta, sigma, x0, T)
variance_POV = calculate_POV_variance(lbd_values, eta, sigma, x0, T)

k_values = np.sqrt(lbd_values / eta) * sigma
expectation_IS = calculate_IS_expectation(k_values, eta, sigma, x0, T)
variance_IS = calculate_IS_variance(k_values, sigma, x0, T)

difference_expectation = np.abs(expectation_POV - expectation_IS) / 100
difference_variance = np.abs(variance_POV - variance_IS) / 10000

df_difference = pd.DataFrame({'Difference_E': difference_expectation, 'Difference_V': difference_variance, 'lambda': lbd_values})

fig = px.scatter(df_difference, x='Difference_V', y='Difference_E', color='lambda', template='plotly_white', 
                 labels={"Difference_V": "Variance Difference", "Difference_E": "Abs(Expectation Difference)", "lambda": "𝛌"}, color_continuous_scale='Blugrn')

fig.update_layout(width=900, height=300)
fig.update_layout(xaxis=dict(range=[0, 5000]))
fig.show()